Things to Clean Up when making Primary/Secondary artists for each row:
- Note that there will be multiple primary and secondary artist columns
- Remove "
- Change '$' to 'S'
- If row contains '(' or ')', make everything primary artist and highlight row red
- Split Primary and Secondary Artists based on 'Featuring', 'Feat.'
- Also Split artists within each category (primary or secondary) based on Comma ',', '&', 'x', 'Duet With', '+', 'And', 'With' - not-case-sensitive
- But only split on 'and' if it's not 'and the', '& the', 'and his', '& his', otherwise just leave original artist unchanged in primary artist column - not-case-sensitive
- But only split on 'with' ir it's not 'with the', otherwise just leave original artist unchanged in primary artist column - not-case-sensitive

Things to Consider
- Values after 'Featuring' are Secondary Artists
- Values before 'Featuring' are Primary Artists
- Just a note, go back and note rows with 'and the', 'with the', '/', 'his orch', '& the' -

Example: 21 Savage, Offset & Metro Boomin Featuring Quavo, Kanye West
- Primary Artist 1: 21 Savage
- Primary Artist 2: Offset
- Primary Artist 3: Metro Boomin
- Secondary Artist 1: Quavo
- Secondary Artist 2: Kanye West

In [4]:
import pandas as pd
import re

# Load your CSV
df = pd.read_csv('billboard_hot_100_unique_artists.csv')  # Replace with your actual file path

# Clean individual artist name
def clean_name(name):
    return name.strip().replace('"', '').replace('$', 'S')

# Check if protected phrase exists (don't split in these cases)
def contains_protected_pattern(text):
    protected = ['and the', '& the', 'and his', '& his', 'with the', 'his orch', '/']
    return any(phrase in text.lower() for phrase in protected)

# Smart splitter for artist blocks (comma, &, x, +, duet with, and, with — case-insensitive)
def smart_split(part):
    if not part or contains_protected_pattern(part):
        return [clean_name(part)]

    split_pattern = r'\s*(,|&|\sx\s|\+|duet with|\band\b|\bwith\b)\s*'
    return [
        clean_name(piece) for piece in re.split(split_pattern, part, flags=re.IGNORECASE)
        if piece.strip() and piece.lower() not in ['and', 'with', 'duet with', '&', '+', ',', ' x ']
    ]

# Full parsing logic
def parse_artist_row(artist_raw):
    raw = str(artist_raw)

    # Highlight if parentheses are present
    if '(' in raw or ')' in raw:
        return {
            'primary': [clean_name(raw)],
            'secondary': [],
            'highlight_red': True
        }

    # Cleanup
    raw = raw.replace('"', '').replace('$', 'S')

    # Split on Featuring / Feat.
    split = re.split(r'(?i)\b(featuring|feat\.?)\b', raw, maxsplit=1)
    primary_part = split[0].strip()
    secondary_part = split[2].strip() if len(split) > 2 else ''

    return {
        'primary': smart_split(primary_part),
        'secondary': smart_split(secondary_part),
        'highlight_red': False
    }

# Apply parsing to each row
parsed = df['Artist'].apply(parse_artist_row)

# Create expanded DataFrame
max_primary = max(parsed.apply(lambda x: len(x['primary'])))
max_secondary = max(parsed.apply(lambda x: len(x['secondary'])))

primary_cols = pd.DataFrame(parsed.apply(lambda x: x['primary']).to_list(), columns=[f'Primary Artist {i+1}' for i in range(max_primary)])
secondary_cols = pd.DataFrame(parsed.apply(lambda x: x['secondary']).to_list(), columns=[f'Secondary Artist {i+1}' for i in range(max_secondary)])
highlight_col = parsed.apply(lambda x: x['highlight_red'])

# Final output
final_df = pd.concat([df, primary_cols, secondary_cols], axis=1)
final_df['Highlight Red'] = highlight_col

# Save or display
final_df.to_csv('parsed_artists_output.csv', index=False)
print("✅ Done! Output saved to 'parsed_artists_output.csv'")


✅ Done! Output saved to 'parsed_artists_output.csv'


Future Fixes:
- Don't split by comma if it's part of a number (e.g. 10,000)
- Only Capitalize 'S' (when replacing a $) if it's at the beginning of a word, lowercase otherwise